<a href="https://colab.research.google.com/github/GeovanaSLima/Geovana-Portfolio/blob/master/Valores_Ausentes_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2 Técnicas para lidar com valores ausentes utilizando o Pandas

Quando lidamos com *datasets* do mundo real, com toda certeza vamos lidar com dados incompletos ou valores ausentes. Como, na maioria dos casos, esses dados são inseridos por pessoas ao longo do tempo, podem existir alterações na base que acabam deixando esses *gaps* no *dataset*. Como por exemplo, uma empresa que por vários anos usa uma base de dados e em algum momento decide adicionar uma nova coluna no seu *dataframe*, fazendo com que os valores antigos não possuam essa informação.

Mas antes de dar continuidade, precisamos saber como são caracterizados esses valores.

### **Valores ausentes**
Esses valores são muito comuns e facilmente encontrado na maioria dos *datasets* existentes. Talvez você nunca tenha percebido, mas eles estão lá. 

Existem dois tipos, o ``None`` e ``NaN``. Mas se os dois são valores ausentes, qual a diferença entre eles?

Bom, os dois soam e parecem muito similares, mas na verdade não são. ``None`` é uma classificação do Python que pode ser considerada o equivalente do NULL, ou seja, valores nulos ou valores não existentes em *object arrays*.

Por outro lado, o ``NaN`` (*Not A Number*) é um valor *floating-point* especial e não pode ser convertido para outro tipo de valor. É utilizado para definir valores faltantes em *numerical arrays*.


Existem inúmeras técnicas para resolver esse tipo de problemas, aqui abordaremos as duas principais.

Para entender melhor essas técnicas, iremos fazer uma aplicação em um *dataset* real.

### **Projeto +BIKE**

O projeto do Governo do Distrito Federal, operado pela empresa Serttel, tem como objetivos:


* Introduzir a Bicicleta como modal de Transporte Público saudável e não poluente.
* Combater o sedentarismo da população e promover a prática de hábitos saudáveis.
* Reduzir os engarrafamentos e a poluição ambiental nas áreas centrais das cidades.
* Promover a humanização do ambiente urbano e a responsabilidade social das pessoas.

Para isso, oferecem estações inteligentes alimentadas por energia solar e conectadas a uma central via *wireless*, distribuídas em diferentes pontos da cidade. O usuário pode retirar a Bicicleta na Estação através do aplicativo baixado em seu smartphone, devolvendo em qualquer Estação após o uso.

<p align="center"><img alt="+BIKE" src="https://i.imgur.com/R0PRD0T.png" width="30%"></p>


***Obs.: Informações retiradas do site oficial do [Projeto +Bike]("http://www.maisbikecompartilhada.com.br/home.aspx").***


---------------------------------------------------------------------------
Antes de começar a aplicação dessas técnicas precisamos, primeiro, entender o *dataset* e identificar os valores ausentes.

In [13]:
DATA_PATH = "http://dl.dropboxusercontent.com/s/yyfeoxqw61o3iel/df_rides.csv"

# importar os pacotes
import pandas as pd

# importar o dataset
df = pd.read_csv(DATA_PATH)

# primeiras entradas
df.head()

,user_gender,user_birthdate,user_residence,ride_date,time_start,time_end,station_start,station_end,ride_duration,ride_late
0,M,1971-06-08,NaN,2018-01-01,06:05:18,06:21:33,11 - Rodoviária 2,41 - Instituto de Artes,16.250000,0.0
1,M,1989-02-11,DF,2018-01-01,06:27:01,06:32:17,26 - Ministério da Saude,28 - CNMP - Conselho Nacional do Ministério Pú...,5.266667,0.0
2,M,1968-07-19,NaN,2018-01-01,06:29:33,06:44:57,11 - Rodoviária 2,43 - Biblioteca Central,15.400000,0.0
3,M,1991-12-19,NaN,2018-01-01,06:53:53,06:59:45,10 - Ministério dos Transportes,6 - Rodoviária,5.866667,0.0
4,M,1969-03-03,DF,2018-01-01,06:58:56,17:40:04,15 - Brasil 21,11 - Rodoviária 2,641.133333,1.0


## **Identificando os valores ausentes**

Na tabela acima, podemos ver que existem entradas com valores ``NaN``, ou seja, valores nulos na coluna ``user_residence``. Mas apenas com as informações acima, não podemos admitir que apenas existam nessa coluna, é indispensável identificar a quantidade de valores ausentes e qual a representatividade dessa quantidade frente ao total de entradas do *dataset* completo.

Para identificar esses valores vamos utilizar o método ``isnull``.


In [14]:
# ver a quantidade de valores ausentes
df.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [15]:
# ver a porcentagem de valores ausentes
df.isnull().sum() / df.shape[0]

user_gender       0.001378
user_birthdate    0.000003
user_residence    0.626144
ride_date         0.000000
time_start        0.000000
time_end          0.150650
station_start     0.000000
station_end       0.000000
ride_duration     0.254676
ride_late         0.254676
dtype: float64

Segundo as duas tabelas acima, vemos que existem colunas com quantidades absurdas de valores faltantes:

* ``user_residence`` - 63%
* ``time_end`` - 15%
* ``ride_duration`` - 25%
* ``ride_late`` - 25%

Duas das informações importantes do nosso *dataset* estão com muitos valores ausentes, ``ride_duration`` e ``ride_late``. Vamos ver como funcionam as técnicas para lidar com esses valores.

## **Excluir valores ausentes**

Está é uma primiera opção para lidar com valores do tipo ``NaN`` em um *DataFrame*.Entretanto, ela não tende a ser ideal, porque você elimina dados existentes em outras colunos por conta de uma única célula. 
Está opção deve ser considerada caso a quantidade de dados nulos seja pequena a ponto de não ter representatividade no *dataset*.

É possível excluir tanto linhas com valores ausentes, quanto colunas inteiras. Para que o Pandas saiba se sua intenção é eliminar linhas (``axis=0``) ou colunas (``axis=1``), devemos informar dentro do método ``dropna()``.

In [16]:
# eliminar todas as entradas onde existam valores ausentes em `user_gender`
df_clean = df.dropna(subset=['user_gender'], axis=0)

# comparar o antes e o depois
print("Antes:\t{}".format(df.shape))
print("Depois:\t{}".format(df_clean.shape))

Antes:	(287322, 10)
Depois:	(286926, 10)


## **Preencher valores**
Normalmente esta é a melhor opção quando lidamos com valores ``NaN``, pois ela permite que você mantenha dados existentes em outras células.

Você deve estar se perguntando, "***Mas por qual valor eu substituoo o valor ausente?***"
Bom, a melhor resposta é: depende.

Existem técnicas simples como usar valores mais frequentes, média e mediana, assim como há técnicas mais avançadas que envolvem o uso de modelos de *machine learning* cuja função é dizer qual valor usar nesses campos.

Aqui iremos ficar nas técnicas mais simples. Vamos ver como podemos usar a mediana para preencher os campos ausentes da coluna ``ride_duration``, com o uso da função ``fillna()``.

In [17]:
# antes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration      73064
ride_late          73064
dtype: int64

In [18]:
# preencher valores ausentes em `ride_duration` com a mediana
rd_median = df_clean.ride_duration.median()
df_clean = df_clean.fillna({'ride_duration': rd_median})

# ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179818
ride_date              0
time_start             0
time_end           43212
station_start          0
station_end            0
ride_duration          0
ride_late          73064
dtype: int64

Analisando as tabelas, vemos que anteriormente a coluna `ride_duration` possuía 73.064 valores ausentes e após a utilização do ``fillna()`` todos os campos estão completos.


No caso da coluna ``user_gender``, temos uma variável categórica. O ideal é usar o valor mais frequente para o preenchimento dos valores. 

Vamos identificar qual valor tem maior recorrência e substituir diretamente na coluna.

In [19]:
# copiar o dataset
df_clean = df.copy()

# ver valores ausentes
df_clean.isnull().sum()

user_gender          396
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

In [20]:
# ver o valor mais frequente
df_clean.user_gender.value_counts()

M    212608
F     74318
Name: user_gender, dtype: int64

In [21]:
# preencher os valores ausentes de user_gender com 'M'
df_clean = df_clean.fillna({'user_gender': 'M'})

# ver valores ausentes
df_clean.isnull().sum()

user_gender            0
user_birthdate         1
user_residence    179905
ride_date              0
time_start             0
time_end           43285
station_start          0
station_end            0
ride_duration      73174
ride_late          73174
dtype: int64

Como podemos ver, agora a variável ``user_gender`` não possui valores ausentes.

---------------------------------------------------------------------------

## **Mas por que devo utilizar esses métodos?**

Como Cientista de Dados, você tem que ter a capacidade de entender, explorar e tratar seus dados da melhor maneira possível. E é muito complicado conseguir bons resultados em uma Análise que tenham quantidades altas de valores faltantes, essas lacunas podem te levar a conclusões completamente equívocas. 

Ao limpar o seu *dataset* você consegue obter informações mais claras e objetivas. Sem contar o fato de que aumenta muito o desempenho do seu algoritmo de *Machine Learning*.

Com essas 2 técnicas apresentadas acima, você já consegue ter uma boa base de como manipular suas bases de dados e melhorar seus projetos de *Data Science*.

